In [9]:
!ollama stop gpt-oss:20b

In [3]:
from openai import OpenAI
 
client = OpenAI(
    base_url="http://localhost:11434/v1",  # Local Ollama API
    api_key="ollama"                       # Dummy key
)
 
stream = client.chat.completions.create(
    model="gpt-oss:20b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "비오는 날 서울에 뭘하면 재미있게 보낼 수 있을까?"}
    ],
    stream=True
)

for chunk in stream:
    print(chunk.choices[0].delta.content, end="", flush=True)

비오는 날 서울에서도 머무르면서 즐길 수 있는 곳이 많아요! 기분은 조금 우울하지만, 실내에서 할 수 있는 재미있는 활동들을 몇 가지 소개할게요. (※시기나 행사에 따라 영업 여부가 바뀔 수 있으니, 방문 전에는 꼭 홈페이지나 전화로 확인하세요.)

---

## 1️⃣ 문화를 즐길 수 있는 실내 명소

| 장소 | 대표 특징 | 추천 포인트 |
|------|-----------|------------|
| **국립현대미술관** (국립공원) | 현대미술 전시, 예술적 분위기 | 주말에는 ‘수상 작품’ 전시가 특별합니다. |
| **북서울박물관** | 예술과 과학이 결합된 전시 | 가족 단위 방문 시 ‘미술작업체험’이 있어요. |
| **홍대 거리 예술가들 카페** | 예술가들의 전시와 창작 현장 | 카페 내부에 있는 ‘벽화’ 감상 추천! |
| **삼성미술관 현대** (삼성동) | 국내외 현대미술 대형전 | ‘전시 설명 가이드’ 신청하면 심층 이해가 돼요. |

---

## 2️⃣ 맛/음식 탐방

| 장소 | 추천 메뉴 | 분위기 |
|------|-----------|--------|
| **제주도 먹자골목** (남대문시장) | 흑돼지, 해물전 | 전통시장의 활기찬 분위기 |
| **아리랑카페** (한강공원 근처) | 시그니처 티라미수 | 카페 내부가 한옥 스타일 |
| **부산집** (이태원) | 회, 해물뚝배기 | 고급스러운 서양식 한식 레스토랑 |
| **맛집 도토리** (명동) | 비빔국수, 냉면 | 24시간 영업, 비 오는 날도 따뜻한 면 |

> **Tip**: 비오는 날은 “주문형 디저트 마이닝(커스터마이즈)”가 인기입니다! 예를 들어, 커피에 바닐라 크림을 더해 특별한 풍미를 느껴보세요.

---

## 3️⃣ 액티비티 & 체험

| 활동 | 장소 | 비용(정가) | 노트 |
|------|------|-----------|------|
| **에스컬레이터 탈옥** (강남구) | 에스컬레이터 대형 탈출 게임 | ~10,000원 | 실내 미로

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather in a given city",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"]
            },
        },
    }
]
 
response = client.chat.completions.create(
    model="gpt-oss:20b",
    messages=[{"role": "user", "content": "What's the weather in Seoul right now?"}],
    tools=tools, 
    stream=True,
)
 
for chunk in response:
    if chunk.choices[0].delta.tool_calls:
        tool_call = chunk.choices[0].delta.tool_calls[0]
        print(f"Tool call: {tool_call.function.name} with arguments {tool_call.function.arguments}")



Tool call: get_weather with arguments {"city":"Seoul"}


In [ ]:
pip install gpt-oss
python -m gpt_oss.responses_api.serve \
    --inference_backend=ollama \
    --checkpoint gpt-oss:20b

In [ ]:
import asyncio
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
 
set_tracing_disabled(True)
 
@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."
 
 
async def main(model: str, api_key: str):
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=LitellmModel(model="ollama/gpt-oss:120b", api_key=api_key),
        tools=[get_weather],
    )
 
    result = await Runner.run(agent, "What's the weather in Tokyo?")
    print(result.final_output)
 
if __name__ == "__main__":
    asyncio.run(main())